In [ ]:
#Initialize dependencies
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
BASE_FOLDER = '/Users/oblivisheee/Documents/GitHub/novelsdreamer-ru-t4m/'
os.chdir(BASE_FOLDER)
from modules.transformer_custom import Transformer
from modules.regularization import RegularizedDenseLayer
from modules.create_config import transformer_config, metrics_config
from modules.datagen import DataGenerator
from modules.inference import translate
#   @tf.keras.mixed_precision.experimental.policy('mixed_float16') # i'll return later

In [ ]:
session_name='novelsdreamer_test_2'

# Config training

In [ ]:
# Configure the Transformer layer
config = transformer_config()
config_metrics = metrics_config()

embedding_layer = tf.keras.layers.Embedding(config["input_vocab_size"], config["d_model"])
regularized_layer = RegularizedDenseLayer(config["d_model"])

model = Transformer(config["num_layers"], config["d_model"], config["num_heads"], config["dff"],
                          config["input_vocab_size"], config["input_vocab_size"], pe_input=config["maximum_position_encoding"],
                          pe_target=config["maximum_position_encoding"], rate=config["dropout_rate"], embedding=embedding_layer, regularized_layer=regularized_layer)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='auto')
optimizer = tf.keras.optimizers.legacy.Adam(name='Adam',learning_rate=0.001,
                                     epsilon=1e-8, amsgrad=True,
                                     beta_1=0.9, beta_2=0.999)

metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name=config_metrics['accuracy_set']), 
           tf.keras.metrics.MeanSquaredError(name=config_metrics['mean_sq_error']), 
           tf.keras.metrics.Precision(thresholds=config_metrics['thresholds'], name=config_metrics['precision']),]


model.compile(optimizer=optimizer, loss=loss_object, metrics=metrics)

In [ ]:
"""MAIN_DATASET_DIR = 'dataset'
TRAIN_DATASET_DIR = os.path.join(MAIN_DATASET_DIR, 'train')
VALID_DATASET_DIR = os.path.join(MAIN_DATASET_DIR, 'valid')

datagen = DataGenerator(train_dir=TRAIN_DATASET_DIR, valid_dir=VALID_DATASET_DIR)
(train_english, train_russian), (valid_english, valid_russian) = datagen.generate()"""

# Start training

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
epochs = 10
model.fit_model(train_english=train_english, 
                         train_russian=train_russian,
                         valid_english=valid_english,
                         valid_russian=valid_russian,
                         epochs=epochs,
                         session_name=session_name,
                         batch_size=64, gradient_accumulation_steps=5,
                         shuffle=True,
                         save_model_each_epoch=True,
                         model_name='novelsdreamer-ru-t4m')

In [ ]:
model.load_weights('novelsdreamer_test_2/results/final_weights/novelsdreamer-ru-t4m.h5')
translated_text, attention_weights = translate(model=model, text="Hello world", max_length=20)
print(translated_text)
